# PyTorch_Learningn (MIVRC)

![](https://ws1.sinaimg.cn/large/abaebc48ly1fqgrsuu355j209x02wjr7.jpg)

## 课程7：残差卷积神经网络

### 代码示例：构建一个具有N个residual block 的 残差卷积神经网络对CIFAR10数据集进行分类

### CIFAR-10
Cifar-10 是由 Hinton 的两个大弟子 Alex Krizhevsky、Ilya Sutskever 收集的一个用于普适物体识别的数据集。Cifar 是加拿大政府牵头投资的一个先进科学项目研究所。Hinton、Bengio和他的学生在2004年拿到了 Cifar 投资的少量资金，建立了神经计算和自适应感知项目。这个项目结集了不少计算机科学家、生物学家、电气工程师、神经科学家、物理学家、心理学家，加速推动了 Deep Learning  的进程。从这个阵容来看，DL 已经和 ML 系的数据挖掘分的很远了。Deep Learning 强调的是自适应感知和人工智能，是计算机与神经科学交叉；Data Mining 强调的是高速、大数据、统计数学分析，是计算机和数学的交叉。

Cifar-10 由60000张32*32的 RGB 彩色图片构成，共10个分类。50000张训练，10000张测试（交叉验证）。这个数据集最大的特点在于将识别迁移到了普适物体，而且应用于多分类（姊妹数据集Cifar-100达到100类，ILSVRC比赛则是1000类）。

![](https://ws1.sinaimg.cn/large/abaebc48ly1fqiwhpbsuxj20ky0g5k6z.jpg)

In [1]:
#导入需要的库/包
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable

In [2]:
#定义超参
num_epochs = 30
batch_size = 100
learning_rate = 0.001
block_number = 5

In [3]:
#图像预处理

# RandomHorizontalFlip 随机水平翻转
# RandomCrop 随机截取32*32大小的图片
# ToTensor 转换为tensor
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32),
    transforms.ToTensor()
])

In [4]:
#下载 CIRAR-10 数据集
train_dataset = datasets.CIFAR10(root='./data/', train=True, transform=transform, download=True)
test_dataset = datasets.CIFAR10(root='./data/', train=True, transform=transform, download=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
#导入数据集
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [6]:
# 定义 Residual Block （残差块）

class ResidualBlock(nn.Module):
    def __init__(self):
        super(ResidualBlock, self).__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv2 = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(64)
        
    def forward(self, x):
        # 把输入保留，后期要用到
        input = x
        
        # 残差学习
        residual = self.bn(self.conv1(x))
        residual = self.relu(residual)
        residual = self.bn(self.conv1(residual))
        
        # 原始输入和残差相加
        out =  input + residual
        out = self.relu(out)
        return out

In [7]:
# 定义 Residual Net （残差网络）

class ResNet(nn.Module):
    def __init__(self):
        super(ResNet, self).__init__()
        self.relu = nn.ReLU()
        self.bn = nn.BatchNorm2d(64)
        self.conv_input = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        
        self.residual = self.make_layer(ResidualBlock, block_number)
        
        self.conv_output = nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.avg_pool = nn.AvgPool2d(4)
        self.fc = nn.Linear(4096, 10)
    
    def make_layer(self, block, num):
        layers = []
        for _ in range(num):
            layers.append(block())
        return nn.Sequential(*layers)
    
    def forward(self, x):
        out = self.relu(self.bn(self.conv_input(x)))
        
        out = self.residual(out)
        
        out = self.relu(self.bn(self.conv_output(out)))
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        #print(out.size())
        out = self.fc(out)
        return out

In [8]:
resnet = ResNet()
resnet.cuda()

ResNet(
  (relu): ReLU()
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (conv_input): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (residual): Sequential(
    (0): ResidualBlock(
      (relu): ReLU()
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (1): ResidualBlock(
      (relu): ReLU()
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    )
    (2): ResidualBlock(
      (relu): ReLU()
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), 

In [9]:
#定义损失函数
criterion = nn.CrossEntropyLoss()
#定义优化器
optimizer = torch.optim.Adam(resnet.parameters(), lr=learning_rate)

In [10]:
#训练网络
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = Variable(images.cuda())
        labels = Variable(labels.cuda())
        
        #调用残差网络进行前向传播
        outputs = resnet(images)
        
        #调用损失函数，求输出值与真实值之间的差
        loss = criterion(outputs, labels)
        
        #启用优化器并将梯度缓存清空，再进行反向传播
        optimizer.zero_grad()
        loss.backward()
        
        #调用优化函数开始迭代优化
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ("Epoch [%d/%d], Iter [%d/%d] Loss: %.4f" %(epoch+1, num_epochs, i+1, 500, loss.data[0]))

Epoch [1/30], Iter [100/500] Loss: 1.3536
Epoch [1/30], Iter [200/500] Loss: 1.2474
Epoch [1/30], Iter [300/500] Loss: 1.1035
Epoch [1/30], Iter [400/500] Loss: 0.9938
Epoch [1/30], Iter [500/500] Loss: 0.9746
Epoch [2/30], Iter [100/500] Loss: 0.9522
Epoch [2/30], Iter [200/500] Loss: 0.8963
Epoch [2/30], Iter [300/500] Loss: 0.6687
Epoch [2/30], Iter [400/500] Loss: 0.7480
Epoch [2/30], Iter [500/500] Loss: 0.5327
Epoch [3/30], Iter [100/500] Loss: 0.9535
Epoch [3/30], Iter [200/500] Loss: 0.6597
Epoch [3/30], Iter [300/500] Loss: 0.6295
Epoch [3/30], Iter [400/500] Loss: 0.7765
Epoch [3/30], Iter [500/500] Loss: 0.5492
Epoch [4/30], Iter [100/500] Loss: 0.4688
Epoch [4/30], Iter [200/500] Loss: 0.7897
Epoch [4/30], Iter [300/500] Loss: 0.6606
Epoch [4/30], Iter [400/500] Loss: 0.7261
Epoch [4/30], Iter [500/500] Loss: 0.5507
Epoch [5/30], Iter [100/500] Loss: 0.7363
Epoch [5/30], Iter [200/500] Loss: 0.6396
Epoch [5/30], Iter [300/500] Loss: 0.5664
Epoch [5/30], Iter [400/500] Loss:

In [12]:
#网络测试
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.cuda())
    outputs = resnet(images)
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted.cpu() == labels).sum()
    
print('Accuracy of the model on the test images: %d %%' % (100 * correct / total))

Accuracy of the model on the test images: 96 %
